In [1]:
import os
import warnings
import sys

import pandas as pd
import numpy as np

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet

import mlflow
import mlflow.sklearn

In [2]:
data = pd.read_csv("SL132001-data (11).csv")

#data.shape
data.head(100)

,Date,Time GMT -4,Timestamp,Ozone - Low Conc.,Hydrogen Sulfide - Low Conc.,Total VOCs (ppm) - PID,Carbon Dioxide - Low Conc.,Particulate Matter 1,Particulate Matter 2.5,Particulate Matter 10,Temperature (Internal),Humidity (Internal),Temperature (External),Humidity (External),Latitude,Longitude,Unnamed: 16
0,DD/MM/YYYY,HH:MM:SS,Epoch,ppm,ppm,ppm,ppm,ug/m3,ug/m3,ug/m3,C,%,C,%,-,-,NaN
1,26/05/2020,8:51:45,1590497505000,0.210,0.000,0.266,466.607,18.270,19.519,19.759,23.87,45.76,23.25,47.54,43.962193,-79.267792,NaN
2,26/05/2020,8:52:44,1590497564000,0.217,0.001,0.268,467.189,14.326,15.149,15.149,23.81,45.93,23.18,47.75,43.962193,-79.267792,NaN
3,26/05/2020,8:53:44,1590497624000,0.222,0.001,0.269,468.038,10.830,11.453,11.453,23.91,45.94,23.25,47.84,43.962193,-79.267792,NaN
4,26/05/2020,8:54:44,1590497684000,0.209,0.001,0.270,473.823,11.914,12.798,13.038,23.85,45.89,23.31,47.87,43.962193,-79.267792,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,26/05/2020,10:29:18,1590503358000,0.231,0.000,0.271,550.662,5.286,5.590,5.590,23.96,48.54,23.53,51.48,43.962193,-79.267792,NaN
96,26/05/2020,10:30:17,1590503417000,0.227,0.000,0.265,553.301,5.136,5.432,5.432,23.94,48.4,23.52,51.59,43.962193,-79.267792,NaN
97,26/05/2020,10:31:17,1590503477000,0.228,0.000,0.263,548.684,5.522,5.839,5.839,23.83,48.51,23.53,51.68,43.962193,-79.267792,NaN
98,26/05/2020,10:32:16,1590503536000,0.229,0.000,0.260,550.657,6.817,7.409,7.650,23.94,48.69,23.55,51.65,43.962193,-79.267792,NaN


In [12]:
# mlflow server --backend-store-uri mlruns/ --default-artifact-root mlruns/ --host 0.0.0.0 --port 5000
remote_server_uri = "file:/deploy/mlflow/mlruns"  #"http://0.0.0.0:5000" # set to your server URI
mlflow.set_tracking_uri(remote_server_uri)  # or set the MLFLOW_TRACKING_URI in the env

In [13]:

#mlflow.tracking.get_tracking_uri()  #file:/

mlflow.tracking.get_tracking_uri()  

'file:/deploy/mlflow/mlruns'

In [14]:
exp_name = "ElasticNet_wine"
mlflow.set_experiment(exp_name)

<Experiment: artifact_location='file:///deploy/mlflow/mlruns/1', experiment_id='1', lifecycle_stage='active', name='ElasticNet_wine', tags={}>

In [19]:


def evaluation_metrics(act, pred):
    # compute relevant metrics
    rmse = np.sqrt(mean_squared_error(act, pred))
    mae = mean_absolute_error(act, pred)
    #r2 = r2_score(act, pred)
    return rmse, mae  #, r2


def load_data(data_path):
    data = pd.read_csv(data_path)

    
    train = data.loc[:, ["Temperature (Internal)", "Humidity (Internal)"]]
    
    return train  # train_x train_y, test_x, test_y
    
train = load_data("SL132001-data (11).csv")
train.head(10)
#train.shape

,Temperature (Internal),Humidity (Internal)
0,C,%
1,23.87,45.76
2,23.81,45.93
3,23.91,45.94
4,23.85,45.89
5,24,46.43
6,23.91,46.33
7,23.86,46.31
8,23.92,46.42
9,23.91,46.25


In [22]:


    


def train(alpha=0.5, l1_ratio=0.5):
    # train a model with given parameters
    warnings.filterwarnings("ignore")
    np.random.seed(40)

    # Read the wine-quality csv file (make sure you're running this from the root of MLflow!)
    #data_path = "SL132001-data (11).csv"
    #train_x, train_y, test_x, test_y = load_data(data_path)
    
    train = load_data("SL132001-data (11).csv")

    # Useful for multiple runs (only doing one run in this sample notebook)    
    with mlflow.start_run():
        # Execute ElasticNet
        lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
        lr.fit(train_x, train_y)

        # Evaluate Metrics
        predicted_qualities = lr.predict(test_x)
        (rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)

        # Print out metrics
        print("Elasticnet model (alpha=%f, l1_ratio=%f):" % (alpha, l1_ratio))
        print("  RMSE: %s" % rmse)
        print("  MAE: %s" % mae)
        print("  R2: %s" % r2)

        # Log parameter, metrics, and model to MLflow
        mlflow.log_param(key="alpha", value=alpha)
        mlflow.log_param(key="l1_ratio", value=l1_ratio)
        mlflow.log_metric(key="rmse", value=7)
        mlflow.log_metrics({"mae": mae, "r2": 8})
        mlflow.log_artifact(data_path)
        print("Save to: {}".format(mlflow.get_artifact_uri()))
        
        mlflow.sklearn.log_model(lr, "model")
        
train()

NameError: name 'train_x' is not defined

In [ ]:
#for loop to input continuous data


In [16]:
data.head(5)

,Date,Time GMT -4,Timestamp,Ozone - Low Conc.,Hydrogen Sulfide - Low Conc.,Total VOCs (ppm) - PID,Carbon Dioxide - Low Conc.,Particulate Matter 1,Particulate Matter 2.5,Particulate Matter 10,Temperature (Internal),Humidity (Internal),Temperature (External),Humidity (External),Latitude,Longitude,Unnamed: 16
0,DD/MM/YYYY,HH:MM:SS,Epoch,ppm,ppm,ppm,ppm,ug/m3,ug/m3,ug/m3,C,%,C,%,-,-,NaN
1,26/05/2020,8:51:45,1590497505000,0.210,0.000,0.266,466.607,18.270,19.519,19.759,23.87,45.76,23.25,47.54,43.962193,-79.267792,NaN
2,26/05/2020,8:52:44,1590497564000,0.217,0.001,0.268,467.189,14.326,15.149,15.149,23.81,45.93,23.18,47.75,43.962193,-79.267792,NaN
3,26/05/2020,8:53:44,1590497624000,0.222,0.001,0.269,468.038,10.830,11.453,11.453,23.91,45.94,23.25,47.84,43.962193,-79.267792,NaN
4,26/05/2020,8:54:44,1590497684000,0.209,0.001,0.270,473.823,11.914,12.798,13.038,23.85,45.89,23.31,47.87,43.962193,-79.267792,NaN
